## first model feature reduction

The next step is to reduce feature size.

By reducing redundancy in data which is fed into the model it is expected to converge better, since less weights have to be adjusted.

In [1]:
from os.path import join

raw = join('data', 'raw')
processed = join('data', 'processed')

from src.training_env import reset_and_populate

reset_and_populate(raw, processed, [400,0,100])

['data\\processed\\train\\n',
 'data\\processed\\validation\\n',
 'data\\processed\\test\\n',
 'data\\processed\\train\\o',
 'data\\processed\\validation\\o',
 'data\\processed\\test\\o',
 'data\\processed\\train\\x',
 'data\\processed\\validation\\x',
 'data\\processed\\test\\x']

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def create_generator(data_dir, batch_size, datagen):
    full_path = join(processed, data_dir)
    return datagen.flow_from_directory(
        full_path,
        target_size=(32, 32),
        color_mode='grayscale',
        batch_size=batch_size,
        class_mode='binary')

train_datagen = ImageDataGenerator(
        rescale = 1./255,
        rotation_range=360,
        horizontal_flip=True,
        vertical_flip=True)

test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = create_generator('train', 20, train_datagen)
test_generator = create_generator('test', 10, test_datagen)

Found 1200 images belonging to 3 classes.
Found 300 images belonging to 3 classes.


In [3]:
from tensorflow.keras import layers
from tensorflow.keras import models

model = models.Sequential()
model.add(layers.Conv2D(8, (2,2), activation='relu', input_shape=(32, 32, 1)))
model.add(layers.Conv2D(16, (2,2), activation='relu'))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Conv2D(32, (2,2), activation='relu'))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Flatten())
model.add(layers.Dense(32,'relu'))
model.add(layers.Dense(3, 'softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 31, 31, 8)         40        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 16)        528       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 32)        2080      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1568)              0         
_________________________________________________________________
dense (Dense)                (None, 32)                5

In [4]:
from tensorflow.keras.optimizers import SGD, RMSprop

optimizer = SGD(lr=0.005, momentum=0.9, nesterov=True)

model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['acc'])

In [5]:
from tensorflow.keras.callbacks import TensorBoard
import numpy as np
from datetime import datetime
from os import mkdir

log_dir = join('logs', datetime.now().strftime("%Y-%m-%dT%H-%M-%S"))
mkdir(log_dir)

from src.training_env import reset
reset(log_dir)

callbacks = [ TensorBoard(
    log_dir=log_dir,
    histogram_freq=1,
    embeddings_freq=1) ]

history = model.fit_generator(
    train_generator,
    steps_per_epoch=20,
    epochs=20,
    callbacks=callbacks)

Epoch 1/20
20/20 [==============================] - 3s 138ms/step - loss: 1.0897 - acc: 0.3500
Epoch 2/20
20/20 [==============================] - 2s 83ms/step - loss: 1.0611 - acc: 0.5550
Epoch 3/20
20/20 [==============================] - 2s 85ms/step - loss: 1.0251 - acc: 0.5525
Epoch 4/20
20/20 [==============================] - 2s 78ms/step - loss: 0.9658 - acc: 0.6175
Epoch 5/20
20/20 [==============================] - 1s 73ms/step - loss: 0.8942 - acc: 0.6875
Epoch 6/20
20/20 [==============================] - 1s 71ms/step - loss: 0.7346 - acc: 0.7525
Epoch 7/20
20/20 [==============================] - 2s 77ms/step - loss: 0.6281 - acc: 0.7925
Epoch 8/20
20/20 [==============================] - 2s 81ms/step - loss: 0.5697 - acc: 0.7625
Epoch 9/20
20/20 [==============================] - 2s 76ms/step - loss: 0.5156 - acc: 0.7900
Epoch 10/20
20/20 [==============================] - 2s 85ms/step - loss: 0.3867 - acc: 0.8650
Epoch 11/20
20/20 [==============================] - 2s 96

In [6]:
model.evaluate_generator(test_generator)

[0.30849321695665516, 0.91333336]

In [7]:
model_path = join('models', 'symbol_classifier', 'first_model_cnn_pooling.h5')
model.save(model_path)